# Chat with LLMs in Amazon Bedrock

Building a chatbot using Foundation Models available in Amazon Bedrock.

## LangChain

LangChain helper utilities used to handle remembering previous chat messages. 

## Import Libraries

Import libraries and create the Bedrock service client.
(We can ignore any warnings that appear when running this cell)

In [ ]:
import json
import os
import sys
import warnings

import boto3

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

In [ ]:
# format instructions into a conversational prompt
from typing import Dict, List

def format_instructions(instructions: List[Dict[str, str]]) -> List[str]:
    """Format instructions where conversation roles must alternate system/user/assistant/user/assistant/..."""
    prompt: List[str] = []
    for instruction in instructions:
        if instruction["role"] == "system":
            prompt.extend(["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n", (instruction["content"]).strip(), " <|eot_id|>"])
        elif instruction["role"] == "user":
            prompt.extend(["<|start_header_id|>user<|end_header_id|>\n", (instruction["content"]).strip(), " <|eot_id|>"])
        else:
            raise ValueError(f"Invalid role: {instruction['role']}. Role must be either 'user' or 'system'.")
    prompt.extend(["<|start_header_id|>assistant<|end_header_id|>\n"])
    return "".join(prompt)

## Use LangChain to maintain chat history

This enables conversational memory capabilities by using the InMemoryChatMessageHistory class from LangChain. InMemoryChatMessageHistory stores conversations, so that the chatbot agent can remember the context from a previous conversation.

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_aws import ChatBedrock

chat_model=ChatBedrock(
    model_id="meta.llama3-8b-instruct-v1:0" , 
    client=bedrock_client)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer the following questions as best you can."),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

history = InMemoryChatMessageHistory()


def get_history():
    return history


chain = prompt | chat_model | StrOutputParser()

wrapped_chain = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key="chat_history",
)
query="What is LangChain?"
response=wrapped_chain.invoke({"input": query})
print(response)

### Chatting with the model

Let's ask a question

In [ ]:
# my question
instructions = [{"role": "user", "content": "Write a bio for Faye Ellis of Pluralsight"}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
print(response)

### Ask a further question

Ask a further question without referencing the previous one, to see if the conversation history is working as expected.

In [ ]:
# asking a further question
instructions = [{"role": "user", "content": "Include my patents"}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
print(response)

### End the conversation

Finish the chat, after that print out the history

In [ ]:
# end the conversation
instructions = [{"role": "user", "content": "That's all, thank you!"}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
print(response)

In [ ]:
print(history)

### Experimenting with Bedrock

- Experiment with different prompts appropriate for your own use case.
- Try using different models to compare how they behave.
- Use prompt engineering to get better outputs.